In [1]:
from glob import glob
import pandas as pd
import numpy
import requests


In [60]:
def get_rainbow_fart():
    response = requests.get("https://api.muxiaoguo.cn/api/caihongpi")
    fart = "何德瑞，"+response.json()["data"]["comment"]
    while len(fart) < 15:
        response = requests.get("https://api.muxiaoguo.cn/api/caihongpi")
        fart = "；" + fart + response.json()["data"]["comment"]
    fart = fart + "。@青春有你-何德瑞"
    return fart


In [63]:
import time

In [64]:


file1 = open("rainbow_fart.txt","w")
for i in range(100):
    time.sleep(2)
    fart = get_rainbow_fart()
 
    
# \n is placed to indicate EOL (End of Line)
    file1.write(fart)
    file1.write("\n")
file1.close() #to change file access modes


TypeError: list indices must be integers or slices, not str

In [65]:
file1.close()

In [19]:
file_name = "stocktakes/V6*.csv"
df = pd.DataFrame()
df_list = []
for f in glob(file_name):
    print(f)
    sub_df = pd.read_csv(f)
    df_list.append(sub_df)
df = pd.concat(df_list,axis=0)
    
    
df["size"] = df.SKU.apply(lambda x:"".join(list(x)[-3:]))


def remove_sizes(string):
    string = string.replace("MED","")
    string = string.replace("XXL","")
    string = string.replace("SML","")
    string = string.replace("MED","")
    string = string.replace("XXS","")
    string = string.replace("LRG","")
    string = string.replace("XLG","")
    string = string.replace("XSM","")
    return string
    
    
df["SKU_noSize"] = df.SKU.apply(lambda x:remove_sizes(x))
minus = df.loc[df.Adjustment < 0]
plus = df.loc[df.Adjustment >0]
print(minus.Adjustment.sum())
print(plus.Adjustment.sum())
display(minus.groupby(["SKU_noSize","size"]).agg({"Adjustment":"sum"}))
display(plus.groupby(["SKU_noSize","size"]).agg({"Adjustment":"sum"}))
pd.set_option("display.max_rows", None, "display.max_columns", None)
display(minus[["SKU","Adjustment","Actual Stock"]])
display(plus[["SKU","Adjustment","Actual Stock"]])

stocktakes/V6_1300.csv.csv
stocktakes/V6_1296.csv
-3
5


,,Adjustment
SKU_noSize,size,
V164LTG,XXL,-1
VW104DSK,XSM,-1
VW136LHG,XSM,-1


Adjustment
SKU_noSize size            
VW104OSH   LRG            1
           XLG            1
VW104SIE   XSM            1
VW158ORE   MED            1
           XSM            1

,SKU,Adjustment,Actual Stock
7,VW136LHGXSM,-1,4
16,VW104DSKXSM,-1,2
8,V164LTGXXL,-1,1


,SKU,Adjustment,Actual Stock
17,VW104OSHLRG,1,4
20,VW104OSHXLG,1,2
26,VW104SIEXSM,1,3
20,VW158OREMED,1,4
22,VW158OREXSM,1,4


## plus["SKU_noSize"].nunique()

In [14]:
file_name = "nightly_logs/v5.csv"
delimiter = ","
dates = ["5/3/2021"]

df2 = pd.read_csv(file_name,delimiter=delimiter)
df2.columns = df2.iloc[0]

def apply_numeric_size(string):
    if string!=string:
        return "ONS"
    if string.isnumeric():
        string = string.zfill(3)
    return string

df2=df2.dropna(subset=["Date","SKU","Style"])
df2 = df2.loc[df2.Date.isin(dates)]
df2.Size = df2.Size.apply(lambda x:apply_numeric_size(x))
df2.Size = df2.Size.str.strip()
df2.SKU = df2.SKU.str.strip()
df2.Color = df2.Color.str.strip()
df2["Notes (LightSpeed Return or Damaged)"] = df2["Notes (LightSpeed Return or Damaged)"].str.strip()
df2.Size = df2.Size.replace({"S":"SML","M":"MED","XS":"XSM","XL":"XLG","L":"LRG"})
df2["SKU_full"] = df2["SKU"] + df2["Color"] + df2["Size"]
df2["SKU_partial"] = df2["SKU"]+df2["Color"]
df2.SKU_full = df2.SKU_full.str.upper()
df2 = df2.loc[df2["Processed\n(Stocky)"] == "FALSE"]
df2["Quantity"] = df2["Quantity"].apply(lambda x:int(x))
print("Return/Damaged/Giftcard and their count of rows")
print(df2.groupby(["Notes (LightSpeed Return or Damaged)"]).Quantity.sum())
damaged = df2.loc[df2["Notes (LightSpeed Return or Damaged)"].isin(["Damage","Dmage","Damage ","Damaged"])]
print("=========")
print("Damaged:")
damaged.groupby(["SKU_partial","Size"]).agg({"Quantity":"sum"})


Return/Damaged/Giftcard and their count of rows
Notes (LightSpeed Return or Damaged)
Damaged    19
Name: Quantity, dtype: int64
Damaged:


,,Quantity
SKU_partial,Size,
V109HCL,XXL,1
V109HMI,XXL,1
V330BKL,LRG,1
V367BCL,LRG,1
V378SAL,LRG,1
V412TOB,LRG,1
V416HCC,LRG,1
V416HPT,LRG,1
V430BLK,032,1


In [16]:
#damaged["SKU_full"] = damaged.SKU_full.apply(lambda x:x.replace(" ",""))
into_csv = damaged.groupby("SKU_full").agg({"Quantity":"sum"}) * -1
into_csv.to_csv("nightly_logs/damage_adjustment.csv")
into_csv = into_csv.reset_index()


In [11]:
into_csv

,SKU_full,Quantity
0,V101BLKSML,-1
1,V142HTALRG,-1
2,V344BLKLRG,-1
3,V416CCLLRG,-1
4,V433BLK033,-1
5,VW409BCMXSM,-1


In [17]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

upc_formula = pd.read_csv("upc_formula.csv")
upc_formula = upc_formula.dropna(subset=["UPC"])
upc_formula["UPC"] = upc_formula.UPC.astype(int)
upc_formula["UPC"] = upc_formula.UPC.astype(str)

upc_formula.Size = upc_formula.Size.apply(lambda x:apply_numeric_size(x))
upc_formula.Size = upc_formula.Size.str.strip()
upc_formula.Size = upc_formula.Size.replace({"S":"SML","M":"MED","XS":"XSM","XL":"XLG","L":"LRG"})
upc_formula[["Style#","Clr","Size"]]
upc_formula["SKU_full"] = upc_formula["Style#"]+upc_formula["Clr"]+upc_formula["Size"]
upc_formula = upc_formula.set_index("SKU_full")
formula_dict = upc_formula["UPC"].to_dict()
upc_code = into_csv.reset_index().SKU_full.replace(formula_dict)
into_csv["UPC"] = upc_code
into_csv["Quantity"] = into_csv.Quantity*-1
into_csv[["UPC","Quantity"]].to_csv("nightly_logs/full_circle_damage.txt",sep="\t",index=False,header=None)

In [18]:
into_csv
    

,SKU_full,Quantity,UPC
0,V109HCLXXL,1,843084163687
1,V109HMIXXL,1,843084177134
2,V330BKLLRG,1,843084119448
3,V367BCLLRG,1,843084182336
4,V378SALLRG,1,840159712108
5,V412TOBLRG,1,V412TOBLRG
6,V416HCCLRG,1,843084108732
7,V416HPTLRG,1,843084184200
8,V430BLK032,1,843084137329
9,V965HTILRG,1,840159712962


In [12]:
lightspeed_return = df2.loc[df2["Notes (LightSpeed Return or Damaged)"]=="Lightspeed Return"]
print(lightspeed_return.Quantity.sum())

display(lightspeed_return.groupby(["SKU_partial","Size"]).agg({"Quantity":"sum"}))
display(lightspeed_return.groupby("SKU_full").agg({"Quantity":"sum"}))
#damaged["SKU_full"] = damaged.SKU_full.apply(lambda x:x.replace(" ",""))
into_csv = lightspeed_return.groupby("SKU_full").agg({"Quantity":"sum"})
into_csv.to_csv("nightly_logs/return.csv")

0


,,Quantity
SKU_partial,Size,


,Quantity
SKU_full,


In [13]:

giftcard = df2.loc[df2["Notes (LightSpeed Return or Damaged)"].isin(["Lightspeed Return Gift Card","Lightspeed Return Giftcard","Lightspeed Giftcard","Gift Return","Gift Return ","Gift","Gift REturn "])]
print(giftcard.Quantity.sum())

display(giftcard.groupby(["SKU_partial","Size"]).agg({"Quantity":"sum"}))
display(giftcard.groupby("SKU_full").agg({"Quantity":"sum"}))
into_csv = giftcard.groupby("SKU_full").agg({"Quantity":"sum"})
into_csv.to_csv("nightly_logs/giftcard.csv")

6


,,Quantity
SKU_partial,Size,
V416HCC,XLG,1
VW123WHT,XSM,1
VW133ORE,XSM,1
VW226PHC,SML,1
VW323BLK,LRG,1
VW367BLK,MED,1


,Quantity
SKU_full,
V416HCCXLG,1
VW123WHTXSM,1
VW133OREXSM,1
VW226PHCSML,1
VW323BLKLRG,1
VW367BLKMED,1
